In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import ee
import sys
import contextlib

# Initialize Project

In [3]:
cloud_project = 'ee-vidyasager162'

try:
    ee.Initialize(project=cloud_project)
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

Enter verification code:  4/1AQlEd8yLKTevia9UWE0GPPpgQPsLtcxf5vxNBzaexF-2txgG_0FXgIg5jJg



Successfully saved authorization token.


In [4]:
import json
import random

In [5]:
pb_output_file = '/kaggle/working/pb_grid_search_output.txt'
oo_output_file = '/kaggle/working/oo_grid_search_output.txt'

# Define the region of interest

In [6]:
# Define the region of interest

roi = ee.Geometry.Polygon(
    [[[11.946657505084461, 43.180502006536486],
      [11.946657505084461, 43.08078081925808],
      [12.071626987506336, 43.08078081925808],
      [12.071626987506336, 43.180502006536486]]])

# Load the dataset and feature collections
dataset = ee.Image("users/vidyasager162/DATASET_COMPOSITION")
newfc = ee.FeatureCollection("users/andreatassi23/Training_point")
valpnts = ee.FeatureCollection("users/andreatassi23/valpnts_AT")
# merged = newfc.merge(valpnts)

classes = [0, 1, 2, 3, 4, 5, 6]

# Train and Test Split

In [7]:
# seed = random.randint(0, 1e6)
# newfc = ee.FeatureCollection([])
# train = ee.FeatureCollection([])
def movePointsForClass(classValue):
    global merged, newfc, seed, valpnts
    classPoints = merged.filter(ee.Filter.eq('LULC', classValue))
    # classPointsWithRandom = classPoints.randomColumn('random', seed)
    classPointsWithRandom = classPoints.randomColumn('random')
    pointsToMove = classPointsWithRandom.sort('random').limit(20)

    pointsToMoveIndices = pointsToMove.aggregate_array('system:index').getInfo()
    exclusionFilter = ee.Filter.Not(
        ee.Filter.inList('system:index', pointsToMoveIndices)
    )
    merged = merged.filter(ee.Filter.Or(
        ee.Filter.neq('LULC', classValue),
        exclusionFilter
    ))

    newfc = newfc.merge(pointsToMove)

# for classValue in classes:
#   movePointsForClass(classValue)

# valpnts = merged

print(dataset.getInfo())
print("Newfc: ", newfc.getInfo())
print("Newfc Size: ", newfc.size().getInfo(), "\n")
print("ValPoints: ", valpnts.getInfo(), "\n")

def movePoints(classValue):
    global newfc, train
    classPoints = newfc.filter(ee.Filter.eq('LULC', classValue))
    classPointsWithRandom = classPoints.randomColumn('random')
    pointsToMove = classPointsWithRandom.sort('random').limit(20)

    train = train.merge(pointsToMove)

# for classValue in classes:
#   movePoints(classValue)

train = newfc

{'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1392, 1111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 11.946605131977105, 0, -8.983152841195215e-05, 43.180578403170806]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1392, 1111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 11.946605131977105, 0, -8.983152841195215e-05, 43.180578403170806]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1392, 1111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 11.946605131977105, 0, -8.983152841195215e-05, 43.180578403170806]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1392, 1111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 11.946605131977105, 0, -8.983152841195215e-05, 43.180578403170806]}, {'id': 'B8', 'data_type': {'type': 'PixelTyp

In [8]:
print("Training: ", train.getInfo(), "\n")
print("Training Size: ", train.size().getInfo(), "\n")
print("Validation Size: ", valpnts.size().getInfo(), "\n")



# Buffer function for training data
buffer = lambda feature: feature.buffer(10)
train = train.map(buffer)

print(train.getInfo(), "\n")

Training:  {'type': 'FeatureCollection', 'columns': {'LULC': 'Integer', 'system:index': 'String'}, 'version': 1604333944433923, 'id': 'users/andreatassi23/Training_point', 'properties': {'system:asset_size': 12287}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [12.029125353233061, 43.12594723264209]}, 'id': '00000000000000000000', 'properties': {'LULC': 1}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [12.046720644370756, 43.12720009512873]}, 'id': '00000000000000000001', 'properties': {'LULC': 1}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [12.042632957832058, 43.12249389758062]}, 'id': '00000000000000000002', 'properties': {'LULC': 1}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [12.011015077964508, 43.11514022859754]}, 'id': '00000000000000000003', 'properties': {'LULC': 1}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [11.981713306355134, 43.121393135973676]}, 'id

# Variables Initialization

In [9]:
# Define classifier parameters
# numTrees = 50

classifiers = {
    'RF': {
        'num_trees_options': [10, 20, 50, 100, 150, 200, 500],
        'max_depth_options': [10, 20, 30, 40, 50, 60, None]
    },
    'SVM': {
        'gamma_options': [0.1, 0.2, 0.25, 0.5, 1],
        'cost_options': [10, 20, 30, 40, 50]
    }
}
# num_trees_options = [10, 20, 50, 100, 150, 200, 500]
# max_depth_options = [10, 20, 30, 40, 50, 60, None]

# gamma_options = [0.1, 0.2, 0.25, 0.5, 1]
# cost_options = [1, 20, 30, 40, 50]

best_accuracy_rf_pb = 0
best_accuracy_svm_pb = 0
best_accuracy_rf_oo = 0
best_accuracy_svm_oo = 0

best_params_rf_pb = {}
best_params_svm_pb = {}
best_params_rf_oo = {}
best_params_svm_oo = {}

best_metrics_rf_pb = ee.Dictionary()
best_metrics_svm_pb = ee.Dictionary()
best_metrics_rf_oo = ee.Dictionary()
best_metrics_svm_oo = ee.Dictionary()

# Helper Functions

In [10]:
# Helper Functions
def unitScale(band):
    band = ee.String(band)
    min_val = ee.Number(minMax.get(band.cat('_min')))
    max_val = ee.Number(minMax.get(band.cat('_max')))
    return image.select(band).unitScale(min_val, max_val)

def compute_metrics(testAccuracy):
    precision = testAccuracy.producersAccuracy()
    recall = testAccuracy.consumersAccuracy().toList().get(0)
    recall = ee.Array(recall)
    f1 = testAccuracy.fscore(1)

    avg_precision = precision.reduce(ee.Reducer.mean(), [0]).toList().get(0)
    avg_precision = ee.Array(avg_precision).get([0])
    avg_recall = recall.reduce(ee.Reducer.mean(), [0]).toList().get(0)
    avg_f1 = f1.reduce(ee.Reducer.mean(), [0]).toList().get(0)

    metrics = ee.Dictionary({
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'avg_precision': avg_precision,
        'avg_recall': avg_recall,
        'avg_f1': avg_f1,
        'overall_accuracy': testAccuracy.accuracy()
    })

    return metrics

def save_metrics(testAccuracy, approach, classifier):
    global best_metrics_rf_pb, best_metrics_svm_pb, best_metrics_rf_oo, best_metrics_svm_oo 
    if approach == 'PIXEL':
        if classifier == 'RF':
            best_metrics_rf_pb = compute_metrics(testAccuracy)
        elif classifier == 'SVM':
            best_metrics_svm_pb = compute_metrics(testAccuracy)
    elif approach == 'OBJECT':
        if classifier == 'RF':
            best_metrics_rf_oo = compute_metrics(testAccuracy)
        elif classifier == 'SVM':
            best_metrics_svm_oo = compute_metrics(testAccuracy)
#print(f'{approach} APPROACH: Confusion Matrix ', testAccuracy.getInfo())
#print(json.dumps(metrics.getInfo()), "\n")

# Pixel-based Approach

In [11]:
# Pixel-based approach

# Mandatory normalization of the input bands for SVM
image = dataset
band_names = image.bandNames()
minMax = image.reduceRegion(reducer=ee.Reducer.minMax(), geometry=roi, scale=10, maxPixels=1e10)

dataset = ee.ImageCollection(band_names.map(unitScale)).toBands().rename(band_names)
bands = dataset.bandNames()

training = dataset.select(bands).sampleRegions(
    collection=train,
    properties=['LULC'],
    scale=10
)

testing = dataset.select(bands).sampleRegions(
  collection=valpnts,
  properties=['LULC'],
  scale=10
)

with open(pb_output_file, 'w') as f, contextlib.redirect_stdout(f):

    for classifier, parameter_dict in classifiers.items():
        if classifier == 'RF':
            num_trees_options = parameter_dict['num_trees_options']
            max_depth_options = parameter_dict['max_depth_options']
            for num_trees in num_trees_options:
                for max_depth in max_depth_options:
                    rf_classifier = ee.Classifier.smileRandomForest(
                        numberOfTrees = num_trees,
                        maxNodes = max_depth
                    ).train(
                        features=training,
                        classProperty='LULC',
                        inputProperties=bands
                    )

                    # Classify and validate using RF
                    classified_rf = dataset.select(bands).classify(rf_classifier).clip(roi).focal_mode()
                    classified_test_rf = testing.classify(rf_classifier)
                    testAccuracy_rf = classified_test_rf.errorMatrix('LULC', 'classification')
                    accuracy_rf = testAccuracy_rf.accuracy().getInfo()

                    # Store the best parameters
                    if accuracy_rf > best_accuracy_rf_pb:
                        best_accuracy_rf_pb = accuracy_rf
                        best_params_rf_pb = {
                            'num_trees': num_trees,
                            'max_depth': max_depth
                        }
                        save_metrics(testAccuracy_rf, 'PIXEL', 'RF')
            print('Best RF Parameters PB:', best_params_rf_pb)
            print('Best RF Metrics PB:', json.dumps(best_metrics_rf_pb.getInfo()))
            print('Best RF Accuracy PB:', best_accuracy_rf_pb, "\n")

        elif classifier == 'SVM':
            gamma_options = parameter_dict['gamma_options']
            cost_options = parameter_dict['cost_options']
            for gamma in gamma_options:
                for cost in cost_options:
                    svm_classifier = ee.Classifier.libsvm(
                        kernelType='RBF',
                        gamma=gamma,
                        cost=cost
                    ).train(training,
                            'LULC',
                            bands
                    )

                    # Classify and validate using SVM
                    classified_svm = dataset.select(bands).classify(svm_classifier).clip(roi).focal_mode()
                    classified_test_svm = testing.classify(svm_classifier)
                    testAccuracy_svm = classified_test_svm.errorMatrix('LULC', 'classification')
                    accuracy_svm = testAccuracy_svm.accuracy().getInfo()

                    # Store the best parameters
                    if accuracy_svm > best_accuracy_svm_pb:
                        best_accuracy_svm_pb = accuracy_svm
                        best_params_svm_pb = {
                            'gamma': gamma,
                            'cost': cost
                        }
                        save_metrics(testAccuracy_svm, 'PIXEL', 'SVM')
            print('Best SVM Parameters PB:', best_params_svm_pb)
            print('Best SVM Metrics PB:', json.dumps(best_metrics_svm_pb.getInfo()))
            print('Best SVM Accuracy PB:', best_accuracy_svm_pb, "\n")

# Define algorithm Parameters

In [12]:
# Define algorithm Parameters

size_segmentation_options = [5, 8, 10, 15, 16, 20, 32]
compactness_options = [0, 0.1, 0.5, 1, 2]
connectivity_options = [4, 8]
neighborhoodSize_options = [32, 64, 128, 256]
glcm_size_options = [1, 2]

# # Define the superpixel seed location spacing, in pixels
# size_segmentation = 15

# Define the GLCM indices used in input for the PCA
glcm_bands = [
    'gray_asm',
    'gray_contrast',
    'gray_corr',
    'gray_ent',
    'gray_var',
    'gray_idm',
    'gray_savg'
]

def scale_bands(name):
    name = ee.String(name)
    band = image.select(name)
    min_val = ee.Number(minMax.get(name.cat('_min')))
    max_val = ee.Number(minMax.get(name.cat('_max')))
    return band.unitScale(min_val, max_val)

# Function to return the Principal Components in the region as a new image
def getPrincipalComponents(centered, scale, region):
    arrays = centered.toArray()
    covar = arrays.reduceRegion(reducer=ee.Reducer.centeredCovariance(), geometry=region, scale=scale)
    covarArray = ee.Array(covar.get('array'))
    eigens = covarArray.eigen()
    eigenValues = eigens.slice(1, 0, 1)
    eigenVectors = eigens.slice(1, 1)

    arrayImage = arrays.toArray(1)
    principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

    sdImage = ee.Image(eigenValues.sqrt()).arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

    return principalComponents.arrayProject([0]).arrayFlatten([getNewBandNames('pc')]).divide(sdImage)

def scale_band(name):
    band_name = ee.String(name)
    min_val = ee.Number(minMax.get(band_name.cat('_min')))
    max_val = ee.Number(minMax.get(band_name.cat('_max')))
    band = final_bands.select(band_name)
    return band.unitScale(min_val, max_val).rename(band_name)

# Create and rescale a grayscale image for GLCM
gray = dataset.expression(
    '(0.3 * NIR) + (0.59 * R) + (0.11 * G)', {
        'NIR': dataset.select('B8'),
        'R': dataset.select('B4'),
        'G': dataset.select('B3')
    }).rename('gray')

# Object-based Approach

In [ ]:
# Object-based approach
with open(oo_output_file, 'w') as f, contextlib.redirect_stdout(f):

    for size_segmentation in size_segmentation_options:
        for compactness in compactness_options:
            for connectivity in connectivity_options:
                for neighborhoodSize in neighborhoodSize_options:
                    for glcm_size in glcm_size_options:
                        print(f"Parameters: Seed Spacing: {size_segmentation}")
                        print(f"Compactness: {compactness}")
                        print(f"Connectivity: {connectivity}"), 
                        print(f"neighborhoodSize: {neighborhoodSize}")
                        print(f"glcm_size: {glcm_size} \n")
                        # Segmentation using a SNIC approach
                        seeds = ee.Algorithms.Image.Segmentation.seedGrid(size_segmentation)
                        snic = ee.Algorithms.Image.Segmentation.SNIC(image=dataset, compactness=compactness, connectivity=connectivity, neighborhoodSize=neighborhoodSize, seeds=seeds)
                        glcm = gray.unitScale(0, 0.30).multiply(100).toInt().glcmTexture(size=glcm_size)

                        # Scaling the GLCM bands before PCA
                        image = glcm.select(glcm_bands)
                        minMax = image.reduceRegion(
                            reducer=ee.Reducer.minMax(),
                            geometry=roi,
                            scale=10,
                            maxPixels=1e10
                        )

                        glcm = ee.ImageCollection.fromImages(
                            image.bandNames().map(scale_bands)
                        ).toBands().rename(image.bandNames())

                        # Applying PCA
                        scale = glcm.projection().nominalScale()
                        bandNames = glcm.bandNames()

                        meanDict = glcm.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale= scale, maxPixels=1e16)

                        means = ee.Image.constant(meanDict.values(bandNames))
                        centered = glcm.subtract(means)

                        getNewBandNames = lambda prefix: ee.List.sequence(1, bandNames.length()).map(lambda b: ee.String(prefix).cat(ee.Number(b).int()))

                        pcImage = getPrincipalComponents(centered, scale, roi)

                        clusters_snic = snic.select('clusters')
                        clusters_snic = clusters_snic.reproject(crs=clusters_snic.projection(), scale=10)

                        new_feature = clusters_snic.addBands(pcImage.select("pc1"))

                        new_feature_mean = new_feature.reduceConnectedComponents(reducer=ee.Reducer.mean(), labelBand='clusters')

                        final_bands = new_feature_mean.addBands(snic)
                        predictionBands = final_bands.bandNames().remove('clusters')

                        # Mandatory normalization of the input bands for SVM
                        minMax = final_bands.reduceRegion(reducer=ee.Reducer.minMax(), geometry=roi, scale=10, maxPixels=1e10)

                        # Apply scaling function to each band
                        scaled_bands = ee.ImageCollection.fromImages(
                            final_bands.bandNames().map(scale_band)
                        ).toBands().rename(final_bands.bandNames())

                        # Replace final_bands with scaled_bands
                        final_bands = scaled_bands

                        training_geobia = final_bands.select(predictionBands).sampleRegions(
                            collection=train,
                            properties=['LULC'],
                            scale=10
                        )

                        classifier_geobia = final_bands.select(predictionBands).sampleRegions(
                            collection=valpnts,
                            properties=['LULC'],
                            scale=10
                        )

                        for classifier, parameter_dict in classifiers.items():
                            if classifier == 'RF':
                                num_trees_options = parameter_dict['num_trees_options']
                                max_depth_options = parameter_dict['max_depth_options']
                                for num_trees in num_trees_options:
                                    for max_depth in max_depth_options:
                                        rf_classifier = ee.Classifier.smileRandomForest(
                                            numberOfTrees = num_trees,
                                            maxNodes = max_depth
                                        ).train(
                                            features=training_geobia,
                                            classProperty='LULC',
                                            inputProperties=predictionBands
                                        )

                                        # Classify and validate using RF
                                        classy_RF = final_bands.select(predictionBands).classify(rf_classifier)
                                        classy_RF = classy_RF.reproject(crs=classy_RF.projection(), scale=10)
                                        classificazione = classifier_geobia.classify(rf_classifier)
                                        testAccuracy_rf = classificazione.errorMatrix('LULC', 'classification')
                                        accuracy_rf = testAccuracy_rf.accuracy().getInfo()

                                        # Store the best parameters
                                        if accuracy_rf > best_accuracy_rf_oo:
                                            best_accuracy_rf_oo = accuracy_rf
                                            best_params_rf_oo = {
                                                'num_trees': num_trees,
                                                'max_depth': max_depth,
                                                'seed spacing': size_segmentation,
                                                'compactness': compactness,
                                                'connectivity': connectivity,
                                                'neigborhoodSize': neighborhoodSize,
                                                'glcm_size': glcm_size
                                            }
                                            save_metrics(testAccuracy_rf, 'OBJECT', 'RF')
    #                                 print(f"Parameters: Seed Spacing: {size_segmentation}, Compactness: {compactness}, Connectivity: {connectivity}, neighborhoodSize: {neighborhoodSize}, and glcm_size: {glcm_size}")
    #                                 print('Best RF Parameters OO:', best_params_rf_oo)
    #                                 print('Best RF Metrics OO:', json.dumps(best_metrics_rf_oo.getInfo()))
    #                                 print('Best RF Accuracy OO:', best_accuracy_rf_oo, "\n")
                                print("RF Done")
                            elif classifier =='SVM':
                                gamma_options = parameter_dict['gamma_options']
                                cost_options = parameter_dict['cost_options']
                                for gamma in gamma_options:
                                    for cost in cost_options:
                                        svm_classifier = ee.Classifier.libsvm(
                                            kernelType='RBF',
                                            gamma=gamma,
                                            cost=cost
                                        ).train(training_geobia,
                                                'LULC',
                                                predictionBands
                                        )

                                        # Classify and validate using SVM
                                        classy_SVM = final_bands.select(predictionBands).classify(svm_classifier)
                                        classy_SVM = classy_SVM.reproject(crs=classy_SVM.projection(), scale=10)
                                        classificazione = classifier_geobia.classify(svm_classifier)
                                        testAccuracy_svm = classificazione.errorMatrix('LULC', 'classification')
                                        accuracy_svm = testAccuracy_svm.accuracy().getInfo()

                                        # Store the best parameters
                                        if accuracy_svm > best_accuracy_svm_oo:
                                            best_accuracy_svm_oo = accuracy_svm
                                            best_params_svm_oo = {
                                                'gamma': gamma,
                                                'cost': cost,
                                                'seed spacing': size_segmentation,
                                                'compactness': compactness,
                                                'connectivity': connectivity,
                                                'neigborhoodSize': neighborhoodSize,
                                                'glcm_size': glcm_size
                                            }
                                            save_metrics(testAccuracy_svm, 'OBJECT', 'SVM')
    #                                 print(f"Parameters: Seed Spacing: {size_segmentation}, Compactness: {compactness}, Connectivity: {connectivity}, neighborhoodSize: {neighborhoodSize}, and glcm_size: {glcm_size}")
    #                                 print('Best SVM Parameters OO:', best_params_svm_oo)
    #                                 print('Best SVM Metrics OO:', json.dumps(best_metrics_svm_oo.getInfo()))
    #                                 print('Best SVM Accuracy OO:', best_accuracy_svm_oo, "\n")
                                print("SVM Done")
    print('Best RF Parameters OO:', best_params_rf_oo)
    print('Best RF Metrics OO:', json.dumps(best_metrics_rf_oo.getInfo()))
    print('Best RF Accuracy OO:', best_accuracy_rf_oo, "\n")
    print('Best SVM Parameters OO:', best_params_svm_oo)
    print('Best SVM Metrics OO:', json.dumps(best_metrics_svm_oo.getInfo()))
    print('Best SVM Accuracy OO:', best_accuracy_svm_oo, "\n")